In [1]:
# import the libraries
import cv2 as cv
import face_recognition as fr
import numpy as np
import os
from datetime import datetime
import pyodbc
import csv

In [2]:
path = 'DATASET'
images = []
personName = []
myList = os.listdir(path)
print(myList)

['AKASH 190301120053.jpg', 'BABLU 190301120090.jpg', 'HITISH 190301120041.jpg', 'PRAVEEN 190301120043.jpg', 'RAKESH 190301120079.jpg', 'RAUSHAN 190301120044.jpg', 'UMASHANKAR 190301120039.jpg']


In [3]:
for cu_img in myList:
    current_Img = cv.imread(f'{path}/{cu_img}')
    images.append(current_Img)
    personName.append(os.path.splitext(cu_img)[0])
print(personName)

['AKASH 190301120053', 'BABLU 190301120090', 'HITISH 190301120041', 'PRAVEEN 190301120043', 'RAKESH 190301120079', 'RAUSHAN 190301120044', 'UMASHANKAR 190301120039']


In [4]:
def faceEncodings(images):
    encodeList = []
    for img in images:
        img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        encode  = fr.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


# print(faceEncodings(images))
encodeListKnown = faceEncodings(images)
print('All Encoding Coplete!!!!')

def attendance(name):
    with open('Attendance.csv','r+') as f:
        myDateList = f.readlines()
        nameList = []
        for line in myDateList:
            entry = line.split(',')
            nameList.append(entry[0])
        
        if name not in nameList:
            time_now = datetime.now()
            tStr = time_now.strftime('%H:%M:%S')
            dStr = time_now.strftime('%d/%m/%Y')
            f.writelines(f'\n{name},{tStr},{dStr}')
            

All Encoding Coplete!!!!


In [6]:
cap = cv.VideoCapture(0)

while True:
    ret,frame = cap.read()
    faces = cv.resize(frame,(0,0),None,0.25,0.25)
    faces = cv.cvtColor(faces,cv.COLOR_BGR2RGB)
    
    facesCurrentFrame = fr.face_locations(faces)
    encodesCurrentFrame = fr.face_encodings(faces,facesCurrentFrame)
    
    for encodeFace, faceLoc in zip(encodesCurrentFrame,facesCurrentFrame):
        matches = fr.compare_faces(encodeListKnown,encodeFace)
        faceDis = fr.face_distance(encodeListKnown,encodeFace)
        
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = personName[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),2)
            cv.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),cv.FILLED)
            cv.putText(frame,name,(x1 + 8,y2 - 6),cv.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            attendance(name)
            
    cv.imshow("Camera",frame)
    if cv.waitKey(1) == 13:
        break
        
cap.release()
cv.destroyAllWindows()

In [12]:
# capture through mobile
camera = 'http://10.40.129.110:8080/video'

cap = cv.VideoCapture(0)
cap.open(camera)
print('check===',cap.isOpened())


while True:
    ret,frame = cap.read()
    faces = cv.resize(frame,(20,30),None,0.25,0.25)
    faces = cv.cvtColor(faces,cv.COLOR_BGR2RGB)
    
    facesCurrentFrame = fr.face_locations(faces)
    encodesCurrentFrame = fr.face_encodings(faces,facesCurrentFrame)
    
    for encodeFace, faceLoc in zip(encodesCurrentFrame,facesCurrentFrame):
        matches = fr.compare_faces(encodeListKnown,encodeFace)
        faceDis = fr.face_distance(encodeListKnown,encodeFace)
        
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = personName[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),2)
            cv.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),cv.FILLED)
            cv.putText(frame,name,(x1 + 8,y2 - 6),cv.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            attendance(name)
            
    cv.imshow("Camera",frame)
    if cv.waitKey(1) == 13:
        break
        
cap.release()
cv.destroyAllWindows()

check=== True
